# Importing data to Spatial data

In [12]:
from skimage.io import imread
import pandas as pd
import xarray as xr
xr.set_options(display_style='text')

In [13]:
import sys
sys.path.append('/home/voehring/voehring/notebooks/spatial-data')

In [14]:
import spatial_data

## Importing raw data

Spatial data implements useful accessors that complement `xarray.Dataset` objects. In order to use them we hence need to convert our image data to `xarray` objects. Here, we assume that our data comes in form of a `tiff` files. To read them into our session we use the `skimage` imread function.

In [15]:
test_image = imread('../../tests/test_files/input.tiff')

By inspecting the `shape` of our image we see that it contains 5 channels and 500 pixels in `x,y` directions.

In [16]:
test_image.shape

(5, 500, 500)

To obtain an `xarray` object that is compatible with `spatial_data` we pass the image to the `spatial_data.load_image_data` function. The `channel_coords` argument enables us pass the appropriate channel names.

In [17]:
img = spatial_data.load_image_data(test_image, channel_coords=['Hoechst', 'CD4', 'CD8', 'FOXP3', 'BCL6'])

`spatial_data.load_image_data` returns an xarray object that we can simply inspect by calling in an jupyter cell. Note that the image is stored as the data variable `_image`.

In [18]:
img

<xarray.Dataset>
Dimensions:   (channels: 5, y: 500, x: 500)
Coordinates:
  * channels  (channels) <U7 'Hoechst' 'CD4' 'CD8' 'FOXP3' 'BCL6'
  * y         (y) int64 0 1 2 3 4 5 6 7 8 ... 492 493 494 495 496 497 498 499
  * x         (x) int64 0 1 2 3 4 5 6 7 8 ... 492 493 494 495 496 497 498 499
Data variables:
    _image    (channels, y, x) uint16 10816 12359 14504 10965 ... 147 129 149 59

## Adding a segmentation mask

We may have a segmentation mask for our image ready. To make use of it we can add it to our `img` object.

In [19]:
seg = imread('../../tests/test_files/segmentation.tiff')

It is important to note that the dimensions of the segmentation mask has to match the `x` and `y` dimension of our original image.

In [20]:
seg.shape

(500, 500)

Now, to add the segmentation mask we call the `se.add_segmentation` accessor. This will append the the segmentation mask to our image object and store it under the data variable `_segmentation`. Additionally, it will add an additional data variable `_obs` that is designed contain metdata for each segmentation object, i.e. cell. This table is initialised to contain the `x` and `y` coordinates of each cell.

In [21]:
img = img.se.add_segmentation(seg)

In [22]:
img

<xarray.Dataset>
Dimensions:        (cells: 3640, features: 2, channels: 5, y: 500, x: 500)
Coordinates:
  * cells          (cells) int64 12 77 91 158 ... 218763 218775 218790 218917
  * features       (features) <U2 '_y' '_x'
  * channels       (channels) <U7 'Hoechst' 'CD4' 'CD8' 'FOXP3' 'BCL6'
  * y              (y) int64 0 1 2 3 4 5 6 7 ... 492 493 494 495 496 497 498 499
  * x              (x) int64 0 1 2 3 4 5 6 7 ... 492 493 494 495 496 497 498 499
Data variables:
    _obs           (cells, features) float64 139.8 49.25 47.77 ... 322.0 454.5
    _image         (channels, y, x) uint16 10816 12359 14504 ... 129 149 59
    _segmentation  (y, x) int32 63194 63194 63194 63194 ... 59633 59633 59633

Taking a closer look at the `img` object, we find that new coordinates `cells` and `features` were added.

## Adding cell type annotations

Finally, we may want to add cell type annotations that assign unique labels to each segmented object of our image.

In [25]:
df = pd.read_csv('../../tests/test_files/labels.csv', index_col=0)